In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install sagemaker boto3 awscli

In [ ]:
!aws s3 ls --profile asher

In [ ]:
local_dir = "./data"

In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.local import LocalSession
role = "arn:aws:iam::345594598345:role/service-role/AmazonSageMaker-ExecutionRole-20250707T163996" # get_execution_role() #
print("Role: ", role)
# local_sess = LocalSession()  # for local session
# local_sess.config = {'local': {'local_code': True}}

sess = sagemaker.Session(boto_session=boto3.Session(region_name="ap-southeast-1")) #name of sso session remove profile name in sagemaker studio , profile_name="asher"
# #for remote session
# settings = dict(
#     sagemaker_session=sess,
#     role=role,
#     instance_type="ml.t3.large",
#     dependencies='./requirements.txt',
# )

bucket_name = sess.default_bucket()
print(bucket_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Role:  arn:aws:iam::345594598345:role/service-role/AmazonSageMaker-ExecutionRole-20250707T163996
sagemaker-ap-southeast-1-345594598345


Uploaded training, testing and validation datasets to S3 bucket

In [ ]:
# !aws s3 cp ./data s3://{bucket_name}/prostate158 --recursive --profile asher
# print("Done!")

Upload weights of pre-trained UNet model

In [ ]:
# !aws s3 cp models/tumor.pt s3://{bucket_name}/models/tumor.pt --profile asher

In [3]:
if bucket_name:
    train_channel = f"s3://{bucket_name}/prostate158/prostate158_train/train"
    test_channel = f"s3://{bucket_name}/prostate158/prostate158_test/test"
    model_channel = f"s3://{bucket_name}/models/tumor.pt"
inputs = {"train": train_channel, "test": test_channel, "model": model_channel}
print(inputs)

{'train': 's3://sagemaker-ap-southeast-1-345594598345/prostate158/prostate158_train/train', 'test': 's3://sagemaker-ap-southeast-1-345594598345/prostate158/prostate158_test/test', 'model': 's3://sagemaker-ap-southeast-1-345594598345/models/tumor.pt'}


In [5]:
!pip install munch

  Using cached munch-4.0.0-py2.py3-none-any.whl.metadata (5.9 kB)
Using cached munch-4.0.0-py2.py3-none-any.whl (9.9 kB)


In [6]:
import yaml
from pathlib import Path
from munch import Munch
# config = yaml.safe_load(Path("tumor.yaml").read_text())
config = Munch.fromDict(yaml.safe_load(Path("tumor.yaml").read_text()))


In [9]:
%%time
from sagemaker.pytorch.estimator import PyTorch
import os

image = "763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-training:2.6.0-gpu-py312-cu124-ubuntu22.04-sagemaker"

def run_training():
    estimator = PyTorch(
        image_uri=image,
        entry_point="train_gpt.py",
        source_dir=".",
        role=role,
        region="ap-southeast-1",
        instance_type="ml.g4dn.2xlarge", # ml.g5.xlarge ran out of memory halfway through 2nd epoch (free tier instance)
        instance_count=1,
        framework_version="2.6.0",
        py_version="py312",
        enable_spot_training=True,  # for cost savings
        # max_run=3600,
        input_mode="File",
        sagemaker_session=sess,
    )
    inputs={
        "training": train_channel,
        "model": model_channel,
        "training_config": os.environ.get("SM_CHANNEL_TRAINING_CONFIG", "./tumor.yaml"),
    }
    estimator.fit(inputs)

run_training()

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-07-15-08-01-51-392


2025-07-15 08:01:51 Starting - Starting the training job...
2025-07-15 08:02:14 Starting - Preparing the instances for training...
2025-07-15 08:02:37 Downloading - Downloading input data...
2025-07-15 08:03:08 Downloading - Downloading the training image...
2025-07-15 08:03:38 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-07-15 08:03:59,537 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-07-15 08:03:59,552 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-07-15 08:03:59,561 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-07-15 08:03:59,562 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-07-15 08:04:01,127 sagemaker-training-toolkit INFO     Installing module.
Processing /opt/ml/c

In [12]:
# !aws s3 cp s3://sagemaker-ap-southeast-1-345594598345/pytorch-training-2025-07-15-08-01-51-392/output/model.tar.gz .
!tar -xvzf model.tar.gz


tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
network_tumor_1_key_metric=0.0066.pt
